<a href="https://colab.research.google.com/github/lazlozerv/Kaggle_SQL_Summer_Camp/blob/master/Lesson6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Intro

So far we have learned how to obtain data from a single table. But what if the data we want is spread across multiple tables?

That's where **JOIN** comes in! __JOIN__ is incredibly important in practical SQL workflows.

#### Example

We'll use 2 imaginary tables `pets` and `owners`. They have 3 columns each. To get information that applies a certain pet, we match the `ID` column in the pets table to the PET_ID column in the `owners` table.


Next, we'll learn how to use **JOIN** to create a new table combining information from the 2 tables.


#### JOIN

Using **JOIN**, we can write a query to create a table with just 2 columns.

In [0]:
## Example
query = """
        SELECT p.Name AS Pet_Name,o.Name AS Owner_Name
        FROM `bigquery-public-data.pet_records.pets` AS p
        INNER JOIN `bigquery-public-data.pet_records.owners` AS o
          ON p.ID=o.Pet_ID
        """

In this query, **ON** determines which column in each table to use to combine the tables.

> In general, when you're joining tables, it's a good habit to specify which table each of your columns come from. That way, you don't have to pull up the schema every time you go back to read the query.


The type of **JOIN** we're using today is called an __INNER JOIN__. That means that a row will only be put in the final output table if the value in the columns you're using to combine them shows up in both the tables you're joining. There are other type of __JOIN__, but an __INNER JOIN__ is very widely used, so it's a good one to start with.

#### GitHub

Now we are going to work with github repo info databases-datasets.

In [0]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "github_repos" dataset
dataset_ref = client.dataset("github_repos", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Construct a reference to the "licenses" table
licenses_ref = dataset_ref.table("licenses")

# API request - fetch the table
licenses_table = client.get_table(licenses_ref)

# Preview the first five lines of the "licenses" table
client.list_rows(licenses_table, max_results=5).to_dataframe()

In [0]:
# Construct a reference to the "sample_files" table
files_ref = dataset_ref.table("sample_files")

# API request - fetch the table
files_table = client.get_table(files_ref)

# Preview the first five lines of the "sample_files" table
client.list_rows(files_table, max_results=5).to_dataframe()

In [0]:
# Query to determine the number of files per license, sorted by number of files
query = """
        SELECT L.license, COUNT(1) AS number_of_files
        FROM `bigquery-public-data.github_repos.sample_files` AS sf
        INNER JOIN `bigquery-public-data.github_repos.licenses` AS L 
            ON sf.repo_name = L.repo_name
        GROUP BY L.license
        ORDER BY number_of_files DESC
        """

# Set up the query (cancel the query if it would use too much of 
# your quota, with the limit set to 10 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(query, job_config=safe_config)

# API request - run the query, and convert the results to a pandas DataFrame
file_count_by_license = query_job.to_dataframe()

We'll begin with the **JOIN**. This specifies the sources of data and how to join them. We use __ON__ to specify that we combine the tables by matching the values in the `repo_name` columns in the tables.


Next, we'll talk about **SELECT** and **GROUP_BY**. The **GROUP BY** breaks the data into a different group for each license, before we __COUNT__ the number of rows in the `sample_files` table that corresponds to each license.


Finally, the __ORDER BY__ sorts the results so that licenses with more files appear first.

This query summarizes how many files have been committed under each license.

In [0]:
# Print the DataFrame
file_count_by_license